<p>Expand your discussion post by revising the table you created and give a better explanation of its goals. Are there any other data points (columns/variables) that need to be added? Is there any other table you need to create? Consider normalization techniques and create at least one more table to be joined; Add more records, update at least one record, create counts, sum, and average, delete one or two records, in summary, apply all the statements you learned this week. Submit your explanation and the python code (ipynb). </p>

<hr />
I stuck with my original example and expanded it as requested.  My assignment includes 3 tables, built following a data normalization pattern.  The three tables are:

<ul>
    <li>Users_jking</li>
    <li>TransactionTypes_jking</li>
    <li>UserTransactions_jking</li>
</ul>

To avoid duplication of TSQL execution patterns I created the exec_tsql helper function.  I ran into frustrating issues with the MySQL connector version, executing multi statements and Python 3.7.  I was ultimately able to get it to work with a workaround that didn't require changes to connector version.  I wrote logic to generate random data and fill the user transaction table.  I then illustrate common CRUD operations such as update and delete.  Finally, I wrote an aggregate statistics query to illustrate how to pull all of the data together.

In [1]:
import random
import pandas as pd
from mysql import connector
from mysql.connector import connect
from mysql.connector.connection import MySQLConnection
connection = connect(host='mysql', database='ALY2100', user='ALY2100', password='homework')
print(f'MySQL Connector version: {connector.__version__}')

MySQL Connector version: 2.2.9


In [2]:
def exec_tsql(tsql, connection, values=None, use_multi=False):
    """
    Generic execute tsql helper
    :param tsql: String Structured Query Language statement
    :param connection: MySQLConnection object
    :param values: List of tuples to be used with Data Manipulation Language type statements
    :param use_multi: Boolean value indicating if the tsql statement contains multiple statements
    :return: Boolean value indicating success
    """    
    # Check that the connection parameter is the expected object type
    if not isinstance(connection, MySQLConnection):
        print(f'connection object must be {MySQLConnection}')
        return False
    
    # Open the connection object if it's currently closed
    if not connection.is_connected():
        connection.connect()
        
    # Create the generic cursor object
    cursor = connection.cursor()
    
    # Exectue the table create DDL and commit, rolling back the transaction if failed
    try:
        # If values is not None the use execute manay, passing in tsql and values
        if values:
            cursor.executemany(tsql, values)
        else:
            # execute() returns an iterator when multi=True and queries are not processed, 
            # regardless of the commit(), call until the iterator has been processed.
            # Create statements produce no results and result in an exception processing the iterator!
            # This is related to a bug in the connector module and has been fixed in version 8.0.13 
            # with support of python 3.7.
            # https://stackoverflow.com/questions/54987200/mysql-connector-cursor-execute-proceeds-silently-but-makes-no-changes-despite
            results = cursor.execute(tsql, multi=use_multi)
            try:
                for result in results:
                    pass
            except Exception as e:
                pass
        # Commit the transaction and return True
        connection.commit()
        return True
    except Exception as e:
        # Display the error on exception, rollback the transaction and return False
        print(f'ERROR: {e}')
        connection.rollback()
        return False

In [3]:
# Generate the Users table DDL and create it via the exec_tsql function
table_name = 'Users_jking'
table_ddl = f"""
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name}(
    UserId int NOT NULL AUTO_INCREMENT,
    Username varchar(16) NOT NULL,
    EmailAddress varchar(128) NOT NULL,
    DateCreated timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
    DateModified timestamp NULL,
PRIMARY KEY CLUSTERED ( UserId ASC ));
"""
exec_tsql(tsql=table_ddl, connection=connection, use_multi=True)

True

In [4]:
# Generate the Users data DML and add it via the exec_tsql function
users = [('TheBride', 'Bride26@gmail.com'), ('GogoYubari', 'gogo@yahoo.com'), 
         ('ORenIshii', 'o.ren@outlook.com'), ('Bill', 'Bill@gates.com'), 
         ('HattoriHanzo', 'hh@gmail.com'), ('ElleDriver', 'edriver@yahoo.com')]
dml = f"INSERT INTO {table_name} (UserName, EmailAddress) VALUES (%s, %s)"
exec_tsql(tsql=dml, connection=connection, values=users, use_multi=True)

True

In [5]:
# Generate the TransactionTypes table DDL and create it via the create_table function
table_name = 'TransactionTypes_jking'
table_ddl = f"""
drop table if exists {table_name};
CREATE TABLE {table_name}(
    TransactionTypeId int NOT NULL AUTO_INCREMENT,
    TransactionType varchar(128) NOT NULL,
    DateCreated timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
    DateModified timestamp NULL,
PRIMARY KEY CLUSTERED ( TransactionTypeId ASC ));
"""
exec_tsql(tsql=table_ddl, connection=connection, use_multi=True)

True

In [6]:
# Generate the TransactionTypes data DML and add it via the create_table function
transaction_types = [('Create', ), ('Read', ), ('Update', ), ('Delete', )]
dml = f"INSERT INTO {table_name} (TransactionType) VALUES (%s)"
exec_tsql(tsql=dml, connection=connection, values=transaction_types)

True

In [7]:
# Generate the UserTransactions table DDL and create it via the create_table function
table_name = 'UserTransactions_jking'
table_ddl = f"""
drop table if exists {table_name};
CREATE TABLE {table_name}(
    UserTransactionsId int NOT NULL AUTO_INCREMENT,
    UserId int NOT NULL,
    TransactionTypeId INT NOT NULL,
    TransactionSize int NOT NULL,
    DateCreated timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
    DateModified timestamp NULL,
PRIMARY KEY CLUSTERED ( UserTransactionsId ASC ));
"""
exec_tsql(tsql=table_ddl, connection=connection, use_multi=True)

True

In [8]:
# Generate the UserTransactions data DML and add it via the create_table function
# Generate two lists matching the auto increment PK values and a random list of size values
user_ids = [x for x in range(1,7)]
trans_type_ids = [x for x in range(1,5)]
trans_size = [x for x in range(24, 1026, 2)]

# Generate the insert statement
dml = f"INSERT INTO {table_name} (UserId, TransactionTypeId, TransactionSize) VALUES (%s, %s, %s)"
# Simulate 100 User Transaction records, with random user and transaction type
for i in range(0, 100):
    random.shuffle(user_ids)
    random.shuffle(trans_type_ids)
    random.shuffle(trans_size)
    # Build a list of randomly select tuples for the values
    transaction_types = [(user_ids[0], trans_type_ids[0], trans_size[0])]
    exec_tsql(tsql=dml, connection=connection, values=transaction_types)

In [9]:
########################################################
# Update user Bill to Jking
cursor = connection.cursor(dictionary=True)

# Generate the update statement
update = "UPDATE Users_jking SET Username = 'jking' WHERE Username = 'Bill'"
cursor.execute(f'select * from Users_jking')

# Print the before values
print(pd.DataFrame(cursor.fetchall()).set_index('UserId'))

# Execute the update statement
exec_tsql(tsql=update, connection=connection)

# Print the after values
cursor.execute(f'select * from Users_jking')
print(pd.DataFrame(cursor.fetchall()).set_index('UserId'))

            Username       EmailAddress         DateCreated DateModified
UserId                                                                  
1           TheBride  Bride26@gmail.com 2021-03-20 13:26:41         None
2         GogoYubari     gogo@yahoo.com 2021-03-20 13:26:41         None
3          ORenIshii  o.ren@outlook.com 2021-03-20 13:26:41         None
4               Bill     Bill@gates.com 2021-03-20 13:26:41         None
5       HattoriHanzo       hh@gmail.com 2021-03-20 13:26:41         None
6         ElleDriver  edriver@yahoo.com 2021-03-20 13:26:41         None
            Username       EmailAddress         DateCreated DateModified
UserId                                                                  
1           TheBride  Bride26@gmail.com 2021-03-20 13:26:41         None
2         GogoYubari     gogo@yahoo.com 2021-03-20 13:26:41         None
3          ORenIshii  o.ren@outlook.com 2021-03-20 13:26:41         None
4              jking     Bill@gates.com 2021-03-20 

In [10]:
########################################################
# Delete user Jking
cursor = connection.cursor(dictionary=True)

# Generate the delete statement
update = "DELETE FROM Users_jking WHERE Username = 'jking'"
cursor.execute(f'select * from Users_jking')

# Print the before values
print(pd.DataFrame(cursor.fetchall()).set_index('UserId'))

# Execute the update statement
exec_tsql(tsql=update, connection=connection)

# Print the after values
cursor.execute(f'select * from Users_jking')
print(pd.DataFrame(cursor.fetchall()).set_index('UserId'))

            Username       EmailAddress         DateCreated DateModified
UserId                                                                  
1           TheBride  Bride26@gmail.com 2021-03-20 13:26:41         None
2         GogoYubari     gogo@yahoo.com 2021-03-20 13:26:41         None
3          ORenIshii  o.ren@outlook.com 2021-03-20 13:26:41         None
4              jking     Bill@gates.com 2021-03-20 13:26:41         None
5       HattoriHanzo       hh@gmail.com 2021-03-20 13:26:41         None
6         ElleDriver  edriver@yahoo.com 2021-03-20 13:26:41         None
            Username       EmailAddress         DateCreated DateModified
UserId                                                                  
1           TheBride  Bride26@gmail.com 2021-03-20 13:26:41         None
2         GogoYubari     gogo@yahoo.com 2021-03-20 13:26:41         None
3          ORenIshii  o.ren@outlook.com 2021-03-20 13:26:41         None
5       HattoriHanzo       hh@gmail.com 2021-03-20 

In [11]:
########################################################
# Aggregate statistics
# Generate the SQL
sql = """
select
    u.UserName,
    tt.TransactionType,
    count(1) as TransactionCount,
    sum(ut.TransactionSize) as TotalTransactionSize,
    avg(ut.TransactionSize) as AverageTransactionSize
from Users_jking u
inner join UserTransactions_jking ut on
    ut.UserId = u.UserID
inner join TransactionTypes_jking tt on
    tt.TransactionTypeId = ut.TransactionTypeId
group by u.UserName, tt.TransactionType
"""
# Execute the SQL and return the results as a dataframe for readability
cursor = connection.cursor(dictionary=True)
cursor.execute(sql)
df = pd.DataFrame(cursor.fetchall())
df

,UserName,TransactionType,TransactionCount,TotalTransactionSize,AverageTransactionSize
0,ElleDriver,Create,2,1344,672.0000
1,ElleDriver,Delete,3,1144,381.3333
2,ElleDriver,Read,4,3830,957.5000
3,ElleDriver,Update,6,2960,493.3333
4,GogoYubari,Create,5,3172,634.4000
5,GogoYubari,Delete,5,1924,384.8000
6,GogoYubari,Read,3,1542,514.0000
7,GogoYubari,Update,2,1088,544.0000
8,HattoriHanzo,Create,1,110,110.0000
9,HattoriHanzo,Delete,6,1214,202.3333
